### Goal
Run flows using sdk and allowing for external configuration of RAG parameters. 

### Check Connections

In [46]:
import promptflow

pf = promptflow.PFClient()

# List all the available connections
for c in pf.connections.list():
    print(c.name + " (" + c.type + ")")

azureopenai_local (AzureOpenAI)
aoai_connection (AzureOpenAI)
default_connection (CognitiveSearch)
cosmodb_transcript (Custom)
cosmodb_connection (Custom)
acs_connection (CognitiveSearch)


### Test the flow

In [48]:
output = pf.flows.test(
    "rag-copilot/flow.dag.yaml",
    inputs={
        "chat_history": [],
        "query": "What is the growth rate of Azure ML revenue in FY23Q1?",
        "topK":5
    },
)

print(output)

{'fetched_docs': '{"retrieved_documents": [{"[doc0]": {"ticker": "MSFT", "quarter": "1", "year": "23", "content": "hiring resulting in mid-to- high-single-digits revenue growth or low to mid- teens growth in constant currency. And in Dynamics, we expect revenue growth in the low-double-digits or the low 20s in constant currency driven by continued share gains in Dynamics 365. For Intelligent Cloud we expect revenue to grow between 22 and 24 percent in constant currency or $21.25 to $21.55 billion US dollars. Revenue will continue to be driven by Azure which, as a reminder, can have quarterly variability primarily from our per-user business and from in- period revenue recognition depending on the mix of contracts. "}}, {"[doc1]": {"ticker": "MSFT", "quarter": "1", "year": "23", "content": "contracts. As a reminder, the growing mix of larger long-term Azure contracts, which are more unpredictable in their timing, always drives increased quarterly volatility in our bookings growth rate. M

### Run the flow with the benchmark data file

In [49]:
flow_path = "rag-copilot/flow.dag.yaml"
data_path = "./datasets/evalset.csv"

column_mapping = {
  "chat_history" :"${data.chat_history}",
  "query": "${data.question}",
  "topK":3,
}

run_topk3 = pf.run(flow = flow_path, data = data_path, column_mapping=column_mapping)
pf.stream(run_topk3)
print(run_topk3)

2024-01-10 19:27:29 -0800   15424 execution.bulk     INFO     Process 15424 started.
2024-01-10 19:27:29 -0800    2024 execution.bulk     INFO     Process 2024 started.
2024-01-10 19:27:29 -0800   38292 execution.bulk     INFO     Process 38292 started.
2024-01-10 19:27:29 -0800   26256 execution.bulk     INFO     Process 26256 started.
2024-01-10 19:27:29 -0800   31684 execution.bulk     INFO     Process 31684 started.
2024-01-10 19:27:29 -0800   24376 execution.bulk     INFO     Process 24376 started.
2024-01-10 19:27:29 -0800    7804 execution.bulk     INFO     Process 7804 started.
2024-01-10 19:27:29 -0800    2720 execution.bulk     INFO     Process 2720 started.
2024-01-10 19:27:29 -0800    6464 execution.bulk     INFO     Process 6464 started.
2024-01-10 19:27:29 -0800    4872 execution.bulk     INFO     Process 4872 started.
======= Run Summary =======

Run name: "rag_copilot_variant_0_20240110_192718_765761"
Run status: "Completed"
Start time: "2024-01-10 19:27:18.765761"
Dura

In [20]:
pf.get_details(run_topk3)

,inputs.chat_history,inputs.query,inputs.TOP_K,inputs.line_number,inputs.maxTokens,inputs.topK,outputs.fetched_docs,outputs.log_params,outputs.reply
0,[],What is the quarterly revenue of Microsoft Clo...,3,0,3500,5,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...","According to [doc0], the Microsoft Cloud excee..."
1,[],How many Azure Arc customers does Microsoft cu...,3,1,3500,5,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...","According to the retrieved documents, Microsof..."
2,[],Which companies run SAP on Azure in FY23Q1?,3,2,3500,5,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...","According to the retrieved documents, companie..."
3,[],What is the annual recurring revenue of GitHub...,3,3,3500,5,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...","According to the retrieved documents, the annu..."
4,[],How many monthly active users does Power BI ha...,3,4,3500,5,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...","According to the retrieved documents, Power BI..."
5,[],What is the quarterly revenue of Microsoft Clo...,3,5,3500,5,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",The Microsoft Cloud exceeded $27 billion in qu...
6,[],How many Azure Arc customers does Microsoft cu...,3,6,3500,5,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...","According to the retrieved document, Microsoft..."
7,[],What are some examples of companies using Azur...,3,7,3500,5,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...","According to the retrieved documents, over 200..."
8,[],How many developers are using GitHub Copilot a...,3,8,3500,5,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...","According to the retrieved documents, as of FY..."
9,[],What is the growth rate of Azure Machine Learn...,3,9,3500,5,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",The retrieved documents do not provide a speci...


## Evaluate 

In [50]:
eval_flow_path = "evaluator/eval_multiple_score/"
data_path = "./datasets/evalset.csv"

eval_run_topk3 = pf.run(
    flow = eval_flow_path,
    run = run_topk3,
    data = data_path,
    column_mapping = {
      "question": "${data.question}",
      "answer": "${data.answer}",
      "reply": "${run.outputs.reply}",
      "context": "${run.outputs.fetched_docs}",
      "log_params":  "${run.outputs.log_params}",
    },
    display_name = "eval_multiple_score_topk3"
)
pf.stream(eval_run_topk3)
print(eval_run_topk3)


2024-01-10 19:28:25 -0800   40288 execution.bulk     INFO     Process 40288 started.
2024-01-10 19:28:25 -0800   10248 execution.bulk     INFO     Process 10248 started.
2024-01-10 19:28:25 -0800   15544 execution.bulk     INFO     Process 15544 started.
2024-01-10 19:28:25 -0800   37884 execution.bulk     INFO     Process 37884 started.
2024-01-10 19:28:25 -0800   26964 execution.bulk     INFO     Process 26964 started.
2024-01-10 19:28:25 -0800   39880 execution.bulk     INFO     Process 39880 started.
2024-01-10 19:28:25 -0800    6592 execution.bulk     INFO     Process 6592 started.
2024-01-10 19:28:25 -0800   10924 execution.bulk     INFO     Process 10924 started.
2024-01-10 19:28:25 -0800   15500 execution.bulk     INFO     Process 15500 started.
2024-01-10 19:28:25 -0800   39168 execution.bulk     INFO     Process 39168 started.
======= Run Summary =======

Run name: "eval_multiple_score_variant_0_20240110_192817_783183"
Run status: "Completed"
Start time: "2024-01-10 19:28:17.

In [51]:
pf.get_details(eval_run_topk3)

,inputs.question,inputs.answer,inputs.reply,inputs.context,inputs.log_params,inputs.line_number,outputs.groundedness,outputs.llm_similarity
0,What is the quarterly revenue of Microsoft Clo...,"$25 billion in quarterly revenue, up 24 percen...","According to [doc0], the Microsoft Cloud excee...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",0,8,10
1,How many Azure Arc customers does Microsoft cu...,"More than 8,500 Arc customers, more than doubl...","According to the retrieved documents, Microsof...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",1,10,10
2,Which companies run SAP on Azure in FY23Q1?,"Companies like Chobani, Munich RE, Sodexo, Vol...","According to the retrieved documents, companie...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",2,8,10
3,What is the annual recurring revenue of GitHub...,GitHub is now at $1 billion in annual recurrin...,"According to the retrieved documents, GitHub h...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",3,10,10
4,How many monthly active users does Power BI ha...,Power BI is the market leader in business inte...,"According to the retrieved documents, Power BI...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",4,2,0
5,What is the quarterly revenue of Microsoft Clo...,"$27 billion in quarterly revenue, up 22 percen...","According to the retrieved documents, the Micr...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",5,10,10
6,How many Azure Arc customers does Microsoft cu...,"More than 12,000 Arc customers, double the num...","According to the retrieved document, Microsoft...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",6,10,10
7,What are some examples of companies using Azur...,"Over 200 customers, including KPMG and Al Jaze...","According to the retrieved documents, companie...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",7,10,8
8,How many developers are using GitHub Copilot a...,More than one million people have used GitHub ...,"According to the retrieved documents, GitHub i...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",8,8,7
9,What is the growth rate of Azure Machine Learn...,Azure ML revenue alone has increased more than...,The retrieved documents do not provide any inf...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",9,1,0


In [52]:
pf.get_metrics(eval_run_topk3)

{'llm_similarity': 7.5, 'llm_groundedness': 7.7}

In [24]:
pf.visualize(eval_run_topk3)

The HTML file is generated at 'C:\\Users\\hokhadiv\\AppData\\Local\\Temp\\pf-visualize-detail-uimb97zm.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.


### Multiple runs 
Using config.yaml file

In [2]:
import yaml
import promptflow

pf = promptflow.PFClient()

with open('./runs_config.yaml', 'r') as file:
    runs_config = yaml.safe_load(file)

# Access the contents of the config.yaml file
print(runs_config)


{'Run1': {'maxTokens': 3500, 'topK': 1}, 'Run2': {'maxTokens': 3500, 'topK': 3}}


In [5]:
from datetime import datetime
run_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
def run_dags(flow_path = "rag-copilot/flow.dag.yaml",
              data_path = "./datasets/evalset.csv",
              eval_flow_path = "evaluator/eval_multiple_score/",
              config= None):
    
    column_mapping = {
    "chat_history" :"${data.chat_history}",
    "query": "${data.question}",
    "topK": config["topK"],
    "maxTokens": config["maxTokens"]
    }

    copilot_run_from_config = pf.run(
        flow = flow_path,
        data = data_path,
        column_mapping=column_mapping,
        display_name="copilot_run_from_config_" + run_time)
    # pf.stream(current_run)
    # print(run_topk3)

    eval_run_from_config = pf.run(
    flow = eval_flow_path,
    run = copilot_run_from_config,
    data = data_path,
    column_mapping = {
      "question": "${data.question}",
      "answer": "${data.answer}",
      "reply": "${run.outputs.reply}",
      "context": "${run.outputs.fetched_docs}",
      "log_params":  "${run.outputs.log_params}",
    },
    display_name = "eval_run_from_config_"+ run_time
    )
    return copilot_run_from_config, eval_run_from_config


In [6]:
i = 0 
run_logs = {}
for run in runs_config:
    i+=1 
    print(runs_config[run])
    copilot_run_from_config, eval_run_from_config = run_dags(config=runs_config[run])
    run_logs[i] = eval_run_from_config


{'maxTokens': 3500, 'topK': 1}
{'maxTokens': 3500, 'topK': 3}


In [14]:
for run_id in run_logs:
    pf.visualize(run_logs[run_id])

The HTML file is generated at 'C:\\Users\\hokhadiv\\AppData\\Local\\Temp\\pf-visualize-detail-2vga5nop.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
The HTML file is generated at 'C:\\Users\\hokhadiv\\AppData\\Local\\Temp\\pf-visualize-detail-57v40egd.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
